<a href="https://colab.research.google.com/github/ReynaQuita/NLP/blob/main/Translation_t5_wo_fine_tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers sacrebleu

     |████████████████████████████████| 266kB 5.2MB/s 
     |████████████████████████████████| 2.5MB 33.9MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 122kB 47.3MB/s 
     |████████████████████████████████| 245kB 44.5MB/s 
     |████████████████████████████████| 3.3MB 17.2MB/s 
     |████████████████████████████████| 901kB 45.0MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [2]:
model_checkpoint = "t5-small"

In [ ]:
# from datasets import load_dataset, load_metric

# train_datasets = load_dataset("wmt16", "ro-en", split="train[:1%]")
# val_datasets = load_dataset("wmt16", "ro-en", split="validation[:1%]")


In [3]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
import datasets

test_data = datasets.load_dataset("wmt16", "ro-en", split="test[:5%]")
metric = datasets.load_metric("sacrebleu")
len(test_data)

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


100

In [ ]:
def generate_translation(batch):
    # cut off at BERT max length 512

    english = ["translate English to Romanian: " + ex["en"] for ex in batch["translation"]]
    # romanian = [ex["ro"] for ex in batch["translation"]]

    inputs = tokenizer(english, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    # input_ids = inputs.input_ids.to("cuda")
    # attention_mask = inputs.attention_mask.to("cuda")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_translation"] = output_str


    return batch

In [ ]:
batch_size = 4  # change to 64 for full evaluation

results = test_data.map(generate_translation, batched=True, batch_size=batch_size)

In [ ]:
bleu_score = 0

for i in range(len(results)):
  ref = results["translation"][i]["ro"]
  pred = results["pred_translation"][i]

  bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
  bleu_score  += bleu_metric["score"]


In [ ]:
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 14.28169222623657 (scale 0-100)


In [ ]:
results['translation'][:5]

[{'en': 'UN Chief Says There Is No Military Solution in Syria',
  'ro': 'Șeful ONU declară că nu există soluții militare în Siria'},
 {'en': 'Secretary-General Ban Ki-moon says his response to Russia\'s stepped up military support for Syria is that "there is no military solution" to the nearly five-year conflict and more weapons will only worsen the violence and misery for millions of people.',
  'ro': 'Secretarul General Ban Ki-moon afirmă că răspunsul său la suportul militar al Rusiei pentru Siria este că „nu există o soluție militară” la conflictul care durează de aproape cinci ani iar mai multe arme nu ar face decât să agraveze violența și suferința a milioane de oameni.'},
 {'en': 'The U.N. chief again urged all parties, including the divided U.N. Security Council, to unite and support inclusive negotiations to find a political solution.',
  'ro': 'Șeful ONU a solicitat din nou tuturor părților, inclusiv Consiliului de securitate ONU divizat să se unifice și să susțină negocierile

In [ ]:
results['pred_translation'][:5]

['eful ONU declară că nu există o soluţie militară',
 'Secretarul General Ban Ki-moon declară că răspunsul său la sprijinul',
 'eful U.N. a cerut din nou tuturor partidelor, inclusiv Consiliului',
 'Ban a declarat la o conferinţă de presă susţinută miercuri că intenţion',
 'El şi-a exprimat regretul că diviziunile din consiliu şi dintre poporul si']

In [15]:
english = "translate English to Chinese: " + "how are you?"
    # romanian = [ex["ro"] for ex in batch["translation"]]

inputs = tokenizer(english, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    # input_ids = inputs.input_ids.to("cuda")
    # attention_mask = inputs.attention_mask.to("cuda")
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
outputs = model.generate(input_ids, attention_mask=attention_mask)

output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [16]:
output_str

['Wie sind Sie?']